In [1]:
!unzip "Record.zip"

Archive:  Record.zip
   creating: Record/
   creating: Record/1/
  inflating: Record/1/1.csv          
  inflating: Record/1/10.csv         
  inflating: Record/1/11.csv         
  inflating: Record/1/12.csv         
  inflating: Record/1/13.csv         
  inflating: Record/1/14.csv         
  inflating: Record/1/15.csv         
  inflating: Record/1/16.csv         
  inflating: Record/1/17.csv         
  inflating: Record/1/18.csv         
  inflating: Record/1/19.csv         
  inflating: Record/1/2.csv          
  inflating: Record/1/20.csv         
  inflating: Record/1/3.csv          
  inflating: Record/1/4.csv          
  inflating: Record/1/5.csv          
  inflating: Record/1/6.csv          
  inflating: Record/1/7.csv          
  inflating: Record/1/8.csv          
  inflating: Record/1/9.csv          
   creating: Record/2/
  inflating: Record/2/1.csv          
  inflating: Record/2/10.csv         
  inflating: Record/2/11.csv         
  inflating: Record/2/12.csv         


In [2]:
!pip install -Uqq fastai tsai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.2/324.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 29.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from fastai.data.all import *
from fastai.basics import *
from tsai.all import *

In [4]:
path = Path("Record/")
path.ls()

(#4) [Path('Record/3'),Path('Record/2'),Path('Record/4'),Path('Record/1')]

In [5]:
files = []
for a in path.ls():
  files += a.ls()
files

(#80) [Path('Record/3/7.csv'),Path('Record/3/5.csv'),Path('Record/3/4.csv'),Path('Record/3/13.csv'),Path('Record/3/8.csv'),Path('Record/3/1.csv'),Path('Record/3/15.csv'),Path('Record/3/20.csv'),Path('Record/3/10.csv'),Path('Record/3/19.csv')...]

In [6]:
def resize(data):
    new_idx = np.linspace(0,data.index[-1],301)
    stretchedData = data.reindex(new_idx,method='ffill', limit=1).iloc[1:].interpolate()
    return stretchedData

def get_x_ts(x):
  val = resize(pd.read_csv(x)).values
  #val = (val - val.min(0))/(val.max(0) - val.min(0))
  return val

In [7]:
dblock_ts = DataBlock(blocks=(TSTensorBlock, CategoryBlock),
                      get_x=get_x_ts,
                      get_y=parent_label,
                      splitter=RandomSplitter(valid_pct=0.2),

                      )

dls_ts = dblock_ts.dataloaders(files,bs=4)


In [8]:
a = dls_ts.dataset[1]
a

(array([[-0.18173212, -0.46567348,  0.24134758,  0.19178696, -0.47400886,
          0.27004013],
        [-0.18132756, -0.46490501,  0.24186262,  0.19253596, -0.472435  ,
          0.27115259],
        [-0.180923  , -0.46413654,  0.24237765,  0.19328496, -0.47086114,
          0.27226505],
        ...,
        [-0.17701277, -0.46370667,  0.2029918 ,  0.17247541, -0.5043806 ,
          0.2319335 ],
        [-0.17559689, -0.46368825,  0.20316401,  0.17183462, -0.50461773,
          0.23144807],
        [-0.17418101, -0.46366984,  0.20333622,  0.17119384, -0.50485486,
          0.23096263]]),
 TensorCategory(2))

In [9]:
net = XceptionTimePlus(dls_ts.dataset[0][0].shape[-2], dls_ts.c)
learn_ts = Learner(dls_ts, net, metrics=accuracy).to_fp16()
learn_ts.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,1.249754,1.356539,0.187500,00:05
1,1.141863,0.986334,0.937500,00:01
2,1.058217,0.743697,0.937500,00:01
3,0.999110,0.621523,1.000000,00:01
4,0.948477,0.605404,1.000000,00:01
5,0.889220,0.528050,1.000000,00:02
6,0.828064,0.518155,1.000000,00:02
7,0.804014,0.485692,1.000000,00:01
8,0.797242,0.469457,1.000000,00:01
9,0.780316,0.448401,1.000000,00:01


In [10]:
learn_ts.export("model.pkl")